In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import defaultdict
from joblib import Parallel, delayed

import sqlite3
import sys
import time
import math
import tqdm
import datetime
import os
import pickle
import random

from glicko2 import Player

if os.path.exists('/workspace/data'):
    # Load the dictionary of DataFrames from the pickle
    data_path = '/workspace/data/'
else:
    data_path = '../data/'

## Loading SQLite Database into Pandas DataFrames

The following code connects to an SQLite database (`melee_player_database.db`) and converts each table within the database into a pandas DataFrame. The DataFrames will be stored in a dictionary, where each key corresponds to the table name with `_df` appended, and the values are the respective DataFrames.

### Steps:

1. **Database Connection**: We use the `sqlite3` library to connect to the SQLite database file.
2. **Retrieve Table Names**: A query retrieves all the table names in the database.
3. **Convert Tables to DataFrames**: For each table:
   - The table is loaded into a pandas DataFrame using `pd.read_sql()`.
   - We check each column to see if any data is JSON-formatted (lists or dictionaries). If so, we convert these columns from strings into their corresponding Python objects using `json.loads()`.
4. **Store DataFrames**: The DataFrames are stored in a dictionary, where the key is the table name with a `_df` suffix, and the value is the DataFrame.
5. **Database Connection Closed**: Once all tables are loaded into DataFrames, the database connection is closed.

### Example:
If the database contains a table named `players`, the corresponding DataFrame will be stored in the dictionary with the key `players_df`, and can be accessed as:

```python
players_df = dfs['players_df']


In [5]:
# Function to get the table names
def get_table_names(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return pd.read_sql(query, conn)['name'].tolist()

# Function to load tables into DataFrames
def load_tables_to_dfs(conn):
    table_names = get_table_names(conn)
    dataframes = {}
    
    for table in table_names:
        # Load table into a DataFrame
        df = pd.read_sql(f"SELECT * FROM {table}", conn)
        
        # Detect and convert JSON formatted columns (if any)
        for col in df.columns:
            # Check if any entry in the column is a valid JSON (list or dictionary)
            if df[col].apply(lambda x: isinstance(x, str)).all():
                try:
                    # Try parsing the column as JSON
                    df[col] = df[col].apply(lambda x: json.loads(x) if pd.notnull(x) else x)
                except (json.JSONDecodeError, TypeError):
                    # If it fails, skip the column
                    pass
        
        # Store the DataFrame with table name + '_df'
        dataframes[f"{table}_df"] = df
        
    return dataframes

if os.path.exists(data_path + 'dfs_dict.pkl'):
    cell_has_run = True
    # Load the dictionary of DataFrames from the pickle
    with open(data_path + 'dfs_dict.pkl', 'rb') as f:
        dfs = pickle.load(f)
# Check if the flag variable exists in the global scope so that this code does not run twice
if 'cell_has_run' not in globals():
    path = data_path + "melee_player_database.db"
    
    # Connect to the database
    conn = sqlite3.connect(path)

    # Convert each table into a DataFrame
    dfs = load_tables_to_dfs(conn)

    # Close the connection
    conn.close()

    # Now, you have a dictionary 'dfs' where each key is the table name with '_df' suffix and value is the corresponding DataFrame.
    # For example, to access the DataFrame for a table called 'players':
    # players_df = dfs['players_df']

    dfs['tournament_info_df']['start'] = pd.to_datetime(dfs['tournament_info_df']['start'], unit='s')
    dfs['tournament_info_df']['end'] = pd.to_datetime(dfs['tournament_info_df']['end'], unit='s')

    
    # Set the flag to indicate that the cell has been run
    cell_has_run = True

### Here we adjust the data types of the dataframes so that they are the correct type. (This will be updated as needed.)

In [6]:
dfs['sets_df']['best_of'] = dfs['sets_df']['best_of'].fillna(0).astype(int) 

### Here we make dataframes that we will use and print the head.

The integers in 'characters' count the number of games the player has played that character. (We verify this for Zain below.)

In [7]:
players_df = dfs['players_df']
players_df.head()

,game,player_id,tag,all_tags,prefixes,social,country,state,region,c_country,c_state,c_region,placings,characters,alias
0,melee,Rishi,Rishi,[Rishi],[],{'twitter': []},,,,None,None,None,[{'key': 'mdva-invitational-2017-(challonge-mi...,,None
1,melee,15634,lloD,"[lloD, VGz | lloD, Llod]",[],{'twitter': ['lloD74']},United States,VA,,US,CA,Laurel,[{'key': 'mdva-invitational-2017-(challonge-mi...,"{'melee/peach': 1089, 'melee/falco': 1, 'melee...",None
2,melee,6126,Zain,"[Zain, DontTestMe]",[PG],{'twitter': ['PG_Zain']},United States,VA,,US,CA,Los Angeles,[{'key': 'mdva-invitational-2017-(challonge-mi...,"{'melee/marth': 1065, 'melee/pichu': 1, 'melee...",DontTestMe
3,melee,Chu,Chu,[Chu],[],{'twitter': []},,,,None,None,None,[{'key': 'mdva-invitational-2017-(challonge-mi...,,None
4,melee,5620,Junebug,"[Junebug, LS | VGz Junebug]",[],{'twitter': ['arJunebug']},United States,VA,,US,VA,Richmond,[{'key': 'mdva-invitational-2017-(challonge-mi...,"{'melee/sheik': 46, 'melee/falco': 4, 'melee/g...",None


In [8]:
ranking_df = dfs['ranking_df']
ranking_df.head()

,game,ranking_name,priority,region,seasons,tournaments,icon
0,melee,SSBMRank,0,world,"[2015, 2016, 2017, 2018, 2019]",[],miom


In [9]:
ranking_seasons_df = dfs['ranking_seasons_df']
ranking_seasons_df.head()

,game,ranking_name,season,start,end,total,by_id,by_placing,final,name
0,melee,SSBMRank,2015,1420070400,1451606399,100,"{'6189': 1, '1004': 2, '4465': 3, '1000': 4, '...","{'1': '6189', '2': '1004', '3': '4465', '4': '...",0,None
1,melee,SSBMRank,2016,1451606400,1483228799,100,"{'6189': 1, '1004': 2, '1000': 3, '1003': 4, '...","{'1': '6189', '2': '1004', '3': '1000', '4': '...",0,None
2,melee,SSBMRank,2017,1483228800,1514764799,100,"{'1004': 1, '6189': 2, '1000': 3, '1003': 4, '...","{'1': '1004', '2': '6189', '3': '1000', '4': '...",0,None
3,melee,SSBMRank,2018,1514793600,1546329600,100,"{'1004': 1, '6189': 2, '4465': 3, '15990': 4, ...","{'1': '1004', '2': '6189', '3': '4465', '4': '...",0,None
4,melee,SSBMRank,2019,1546329600,1577836800,100,"{'1004': 1, '4465': 2, '1000': 3, '16342': 4, ...","{'1': '1004', '2': '4465', '3': '1000', '4': '...",0,None


In [10]:
sets_df = dfs['sets_df']
print(f"{sets_df[sets_df['game_data'].apply(lambda x: len(x) > 0)].shape[0] / sets_df.shape[0]:0.01%} percent of sets have some game data)")

sets_df.head()

32.9% percent of sets have some game data)


,key,game,tournament_key,winner_id,p1_id,p2_id,p1_score,p2_score,location_names,bracket_name,bracket_order,set_order,best_of,game_data
0,104675843,melee,mdva-invitational-2017-(challonge-mirror),5620,5620,Chillin,3,1,"[R1, Round 1, Round 1]",,1,A,5,[]
1,104675844,melee,mdva-invitational-2017-(challonge-mirror),Aglet,15634,Aglet,2,3,"[R1, Round 1, Round 1]",,1,B,5,[]
2,104675845,melee,mdva-invitational-2017-(challonge-mirror),6126,6126,1097,3,0,"[R1, Round 1, Round 1]",,1,C,5,[]
3,104675846,melee,mdva-invitational-2017-(challonge-mirror),1069,Chu,1069,0,3,"[R1, Round 1, Round 1]",,1,D,5,[]
4,104675847,melee,mdva-invitational-2017-(challonge-mirror),Rishi,Jerry,Rishi,1,3,"[R1, Round 1, Round 1]",,1,E,5,[]


In [11]:
tournament_info_df = dfs['tournament_info_df']
tournament_info_df.head()

,game,key,cleaned_name,source,tournament_name,tournament_event,season,rank,start,end,country,state,city,entrants,placings,losses,bracket_types,online,lat,lng
0,melee,mdva-invitational-2017-(challonge-mirror),MDVA Invitational 2017 (Challonge Mirror),challonge,https://challonge.com/mdva_invitational_2017,,17,,2017-11-26 08:05:11,2017-11-26 08:48:09,US,VA,Fall's Church,10,"[[Rishi, 1], [15634, 3], [6126, 4], [Chu, 8], ...",{},b'{}',0,NaN,NaN
1,melee,s@sh7,S@SH7,challonge,https://challonge.com/sash7,,17,,2017-06-13 10:27:01,2017-06-13 10:27:01,US,MI,Ann Arbor,92,[],{},b'{}',0,NaN,NaN
2,melee,slippi-champions-league-week-1__melee-singles,Slippi Champions League Week 1,pgstats,slippi-champions-league-week-1,melee-singles,20,,2020-10-11 14:00:00,2020-10-11 14:00:00,,,,20,"[[1000, 1], [6126, 2], [4107, 3], [19554, 3], ...",{},b'{}',1,0.0,0.0
3,melee,slippi-champions-league-week-2__melee-singles,Slippi Champions League Week 2,pgstats,slippi-champions-league-week-2,melee-singles,20,,2020-10-18 14:00:00,2020-10-18 14:00:00,,,,20,"[[6126, 1], [4107, 2], [1000, 3], [19554, 3], ...",{},b'{}',1,0.0,0.0
4,melee,slippi-champions-league-week-3__melee-singles,Slippi Champions League Week 3,pgstats,slippi-champions-league-week-3,melee-singles,20,,2020-10-25 14:00:00,2020-10-25 14:00:00,,,,20,"[[6126, 1], [3359, 2], [19554, 3], [4107, 3], ...",{},b'{}',1,0.0,0.0


In [12]:
# Code optimization by Dan
# Basically we want to replace this line in process_tournament with something more efficient:
#
#      tournament_sets_df = sets_df[sets_df['tournament_key'] == tournament_key]
#
# Instead, we can
# - Merge the tournament date info into ``sets_df``
# - Sort by date
# - Store the start/end positions of each tournament in a separate dictionary
# - Use tournament_sets_df = sets_df.iloc[start:end+1] instead.

sets_df = sets_df.merge(tournament_info_df[['key', 'start', 'end']], left_on='tournament_key', right_on='key', how='left')
sets_df = sets_df.drop(labels=['key_y'], axis='columns')
sets_df = sets_df.rename(columns={"key_x": "key"})
sets_df = sets_df.sort_values(by=['end', 'tournament_key']) # Just in case there are tournaments with the exact same end date

In [13]:
# A bit of data cleanup
min_date = datetime.datetime(2015, 1, 1)
max_date = datetime.datetime(2024, 12, 31)

sets_df = sets_df[(sets_df['start'] >= min_date) & (sets_df['end'] >= min_date) & (sets_df['start'] <= max_date) & (sets_df['end'] <= max_date)]

## Generate features

In [14]:
# Append to this list evey time you add features!
# For convenience, it is blank here, and you should append in each feature-adding cell
features = []
output = 'winner'

In [15]:
sets_df[output] = sets_df.apply(lambda row: 1.0 if row['winner_id'] == row['p1_id'] else 0.0, axis=1)

### "Default" ELO/RD features

In [16]:
# Overall ELO/RD as features
player_ratings_df = pd.read_pickle(data_path + 'overall_players_ranking_new_weekly.pkl')
player_rds_df = pd.read_pickle(data_path + 'overall_players_rds_new_weekly.pkl')

# Might as well compute a (mostly accurate) number of updates from the ratings dataframe
# This isn't 100% accurate, as sometimes (especially if default elo playing default elo)
# the elo might not change from round to round.
player_updates_df = pd.DataFrame(0, index=player_ratings_df.index, columns=player_ratings_df.columns)

for i in range(1, len(player_ratings_df.index)):
    row_updates = (player_ratings_df.iloc[i] != player_ratings_df.iloc[i-1]).astype(int)
    player_updates_df.iloc[i] = row_updates

player_updates_df = player_updates_df.cumsum()


features_elo_rd = ['p1_elo', 'p2_elo', 'p1_rd', 'p2_rd', 'p1_updates', 'p2_updates']
dates = list(player_ratings_df.index)

# Efficiency purposes. We are assuming that constant intervals are used.
initial_date = dates[0]
interval = dates[1] - dates[0]

# start_date is the start date of the specific set (tournament)
def get_info(df, player_id, start_date):
    # Far more efficient than searching the index manually every time
    week_num = int((start_date - initial_date) / interval)
    newest_date = initial_date + week_num * interval
    if dates[-1] < newest_date:
        newest_date = dates[-1]
    return df.loc[newest_date, player_id]

sets_df['p1_elo'] = sets_df.apply(lambda row: get_info(player_ratings_df, row['p1_id'], row['start']), axis=1)
sets_df['p2_elo'] = sets_df.apply(lambda row: get_info(player_ratings_df, row['p2_id'], row['start']), axis=1)

sets_df['p1_rd'] = sets_df.apply(lambda row: get_info(player_rds_df, row['p1_id'], row['start']), axis=1)
sets_df['p2_rd'] = sets_df.apply(lambda row: get_info(player_rds_df, row['p2_id'], row['start']), axis=1)

sets_df['p1_updates'] = sets_df.apply(lambda row: get_info(player_updates_df, row['p1_id'], row['start']), axis=1)
sets_df['p2_updates'] = sets_df.apply(lambda row: get_info(player_updates_df, row['p2_id'], row['start']), axis=1)

# Might as well clear if these aren't needed. They are huge in memory.
del player_ratings_df
del player_rds_df
del player_updates_df

### Number of times each player has played each character

In [17]:
initial_date = datetime.datetime(2015,1,1)
interval = datetime.timedelta(weeks=1)

end_date = initial_date + math.ceil((sets_df['end'].max() - initial_date) / interval) * interval

# Assumes game_data_extractor.ipynb was run
game_data = pd.read_pickle(data_path + 'individual_game_data.pkl')

In [18]:
# Actually get a list of characters and their individual popularities
# In the end, a series with index as the character and value as the number of times played.
# Probably sorted in the end.
all_characters = pd.concat([game_data['winner_char'], game_data['loser_char']], ignore_index=True)
all_characters = all_characters.value_counts()

In [19]:
# Player/character combos
game_data['winner_pc'] = game_data.apply(lambda row: row['winner_id'] + '/' + row['winner_char'], axis=1)
game_data['loser_pc'] = game_data.apply(lambda row: row['loser_id'] + '/' + row['loser_char'], axis=1)

# What week number this is, for example.
game_data['end_index'] = game_data['end'].apply(lambda x: math.ceil((x - initial_date) / interval))
game_data['end_date'] = game_data['end_index'].apply(lambda x: initial_date + x*interval)
game_data['end_date_copy'] = game_data['end_index'].apply(lambda x: initial_date + x*interval) # To deal with include_groups=True deprecation


In [20]:
# Actually compute the dates (index) without loading a dataframe for reference.
dates = []

date = initial_date
while date <= end_date:
    dates.append(date)
    date += interval

pc_combos = list(set(list(game_data['winner_pc']) + list(game_data['loser_pc'])))

# Count the number of times a character has been used.
# Columns will be player/character (or rather, player/melee/character)
# Initialize everything to zero, initially.
character_usage_df = pd.DataFrame(0, index=dates, columns=pc_combos)

In [21]:
def update_row(df):
    date = df.iloc[0]['end_date_copy']

    pc_series = pd.concat([df['winner_pc'], df['loser_pc']], ignore_index=True)

    # Deprecation-proofing
    temp = character_usage_df.loc[date].copy()
    temp.update(pc_series.value_counts())
    
    character_usage_df.loc[date] = temp
    
game_data.groupby('end_date').apply(update_row, include_groups=False)

""


In [22]:
character_usage_df = character_usage_df.cumsum()
character_usage_df

,15989/bowser,3762621/pikachu,525661/falco,151717/kirby,2404168/peach,45659/iceclimbers,500241/fox,491552/falco,2036579/fox,1105705/marth,...,7720/fox,2373292/bowser,41354/sheik,2284148/captainfalcon,14515/falco,2277668/falco,2824998/marth,1615412/falco,2384244/bowser,3544229/sheik
2015-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-17,38,2,66,1,49,3,15,17,5,1,...,16,42,1,2,2,6,1,461,2,107
2024-10-24,38,2,66,1,49,3,15,17,5,1,...,16,42,1,2,2,6,1,461,2,107
2024-10-31,38,2,66,1,49,3,15,17,5,1,...,16,42,1,2,2,6,1,461,2,107
2024-11-07,38,2,66,1,49,3,15,17,5,1,...,16,42,1,2,2,6,1,461,2,107


In [23]:
# Newest possible date to get data from.
sets_df['start_index'] = sets_df['start'].apply(lambda x: int((x - initial_date) / interval))
sets_df['start_date'] = sets_df['start_index'].apply(lambda x: initial_date + x * interval)

In [ ]:
# No point in recomputing every single time.
p1c_final_index = ['p1' + '_' + x + '_count' for x in all_characters.index]
p2c_final_index = ['p2' + '_' + x + '_count' for x in all_characters.index]

features_pc = p1c_final_index + p2c_final_index

def get_char_count(row):
    # We'll rename the index entries later for these.
    p1c_series = pd.Series(0, index=[row['p1_id'] + '/' + x for x in all_characters.index])
    p2c_series = pd.Series(0, index=[row['p2_id'] + '/' + x for x in all_characters.index])

    p1c_series.update(character_usage_df.loc[row['start_date']])
    p2c_series.update(character_usage_df.loc[row['start_date']])

    p1c_series.index = p1c_final_index
    p2c_series.index = p2c_final_index

    return pd.concat([p1c_series, p2c_series])

sets_df = pd.concat([sets_df, sets_df.apply(get_char_count, axis=1)], axis=1)

## Compute most-used character info

In [ ]:
p1_char_usage_df = sets_df[p1c_final_index]
p2_char_usage_df = sets_df[p2c_final_index]

p1_char_usage_df

,p1_fox_count,p1_falco_count,p1_marth_count,p1_sheik_count,p1_captainfalcon_count,p1_jigglypuff_count,p1_peach_count,p1_luigi_count,p1_samus_count,p1_ganondorf_count,...,p1_mario_count,p1_zelda_count,p1_roy_count,p1_younglink_count,p1_kirby_count,p1_ness_count,p1_bowser_count,p1_pichu_count,p1_random_count,p1_mewtwo_count
681,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
682,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
683,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
684,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
685,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489901,0,20,0,21,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1489902,0,0,0,0,296,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
1489903,96,7,7,0,17,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1489904,0,0,0,0,296,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0


In [ ]:
def get_mains(row, player):
    # Compute three mains and their usages
    index = ['m1', 'm2', 'm3', 'm1_usage', 'm2_usage', 'm3_usage']
    index = [player + '_' + x for x in index]

    row_sorted = row.sort_values(ascending=False)

    mains = []

    # First three entries are the actual mains
    for i in range(0,3):
        mains.append(row_sorted.index[i].removeprefix(player + '_').removesuffix('_count'))

    # Next three are how many times they have been played
    for i in range(0,3):
        mains.append(row_sorted.iloc[i])

    return pd.Series(mains, index=index)


p1_mains_df = p1_char_usage_df.apply(lambda row: get_mains(row, 'p1'), axis=1)
p2_mains_df = p2_char_usage_df.apply(lambda row: get_mains(row, 'p2'), axis=1)

In [ ]:
# Temporary workspace for computing individual matchup elos
player_mains_df = pd.concat([sets_df[['p1_id', 'p2_id', 'start']], p1_mains_df, p2_mains_df], axis=1)
alt2_df = pd.read_pickle(data_path + 'char_vs_char_player_rankings_weekly_alt2.pkl')

# How many times each column was updated so far
# Keep everything float to not change the data type
alt2_updates_df = alt2_df.copy()
alt2_updates_df.iloc[0] = 0.0
alt2_updates_df.iloc[1:] = (alt2_updates_df.iloc[1:].reset_index(drop=True) != alt2_updates_df.iloc[:-1].reset_index(drop=True)).astype(float).values
alt2_updates_df = alt2_updates_df.cumsum()

start_date = alt2_df.index[0]
interval = alt2_df.index[1] - alt2_df.index[0]

# Newest possible date to pull matchup info from
player_mains_df['start_index'] = player_mains_df['start'].apply(lambda x: int((x - start_date) / interval))
player_mains_df['newest_date'] = player_mains_df['start_index'].apply(lambda x: start_date + x*interval)

player_mains_df

,p1_id,p2_id,start,p1_m1,p1_m2,p1_m3,p1_m1_usage,p1_m2_usage,p1_m3_usage,p2_m1,p2_m2,p2_m3,p2_m1_usage,p2_m2_usage,p2_m3_usage,start_index,newest_date
681,1000,Stelzig,2015-01-11 14:16:13,fox,falco,marth,0,0,0,fox,falco,marth,0,0,0,1,2015-01-08
682,Flikkflakk,IVP,2015-01-11 14:16:13,fox,falco,marth,0,0,0,fox,falco,marth,0,0,0,1,2015-01-08
683,Humpe,Yomi,2015-01-11 14:16:13,fox,falco,marth,0,0,0,fox,falco,marth,0,0,0,1,2015-01-08
684,Thomas,Hao,2015-01-11 14:16:13,fox,falco,marth,0,0,0,fox,falco,marth,0,0,0,1,2015-01-08
685,12870,JJLinyard,2015-01-11 14:16:13,fox,falco,marth,0,0,0,fox,falco,marth,0,0,0,1,2015-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489901,1602510,7564,2023-11-12 07:30:00,sheik,falco,fox,21,20,0,marth,fox,falco,136,0,0,462,2023-11-09
1489902,698885,25647,2023-11-12 07:30:00,captainfalcon,link,kirby,296,20,2,falco,fox,marth,22,0,0,462,2023-11-09
1489903,10565,1602510,2023-11-12 07:30:00,fox,captainfalcon,falco,96,17,7,sheik,falco,fox,21,20,0,462,2023-11-09
1489904,698885,1602510,2023-11-12 07:30:00,captainfalcon,link,kirby,296,20,2,sheik,falco,fox,21,20,0,462,2023-11-09


In [ ]:
# TODO: Pull number of times this specific matchup happened as well?

matchups = []

# Compute the actual matchups first.
# Then obtain the elos.

for i in [1,2]: # player_i
    for j in [1,2,3]: # main_j
        for k in [1,2,3]: # opponent_main_k
            player_num = i
            opponent_num = 3-i # swap 1 and 2

            player = player_mains_df['p' + str(player_num) + '_id']
            player_main = player_mains_df['p' + str(player_num) + '_m' + str(j)]
            opponent_main = player_mains_df['p' + str(opponent_num) + '_m' + str(k)]

            col_name = 'p' + str(player_num) + '/m' + str(j) + '/m' + str(k)

            matchups.append(col_name)
            player_mains_df[col_name] = player + '/' + player_main + '/' + opponent_main

player_mains_df


,p1_id,p2_id,start,p1_m1,p1_m2,p1_m3,p1_m1_usage,p1_m2_usage,p1_m3_usage,p2_m1,...,p1/m3/m3,p2/m1/m1,p2/m1/m2,p2/m1/m3,p2/m2/m1,p2/m2/m2,p2/m2/m3,p2/m3/m1,p2/m3/m2,p2/m3/m3
681,1000,Stelzig,2015-01-11 14:16:13,fox,falco,marth,0,0,0,fox,...,1000/marth/marth,Stelzig/fox/fox,Stelzig/fox/falco,Stelzig/fox/marth,Stelzig/falco/fox,Stelzig/falco/falco,Stelzig/falco/marth,Stelzig/marth/fox,Stelzig/marth/falco,Stelzig/marth/marth
682,Flikkflakk,IVP,2015-01-11 14:16:13,fox,falco,marth,0,0,0,fox,...,Flikkflakk/marth/marth,IVP/fox/fox,IVP/fox/falco,IVP/fox/marth,IVP/falco/fox,IVP/falco/falco,IVP/falco/marth,IVP/marth/fox,IVP/marth/falco,IVP/marth/marth
683,Humpe,Yomi,2015-01-11 14:16:13,fox,falco,marth,0,0,0,fox,...,Humpe/marth/marth,Yomi/fox/fox,Yomi/fox/falco,Yomi/fox/marth,Yomi/falco/fox,Yomi/falco/falco,Yomi/falco/marth,Yomi/marth/fox,Yomi/marth/falco,Yomi/marth/marth
684,Thomas,Hao,2015-01-11 14:16:13,fox,falco,marth,0,0,0,fox,...,Thomas/marth/marth,Hao/fox/fox,Hao/fox/falco,Hao/fox/marth,Hao/falco/fox,Hao/falco/falco,Hao/falco/marth,Hao/marth/fox,Hao/marth/falco,Hao/marth/marth
685,12870,JJLinyard,2015-01-11 14:16:13,fox,falco,marth,0,0,0,fox,...,12870/marth/marth,JJLinyard/fox/fox,JJLinyard/fox/falco,JJLinyard/fox/marth,JJLinyard/falco/fox,JJLinyard/falco/falco,JJLinyard/falco/marth,JJLinyard/marth/fox,JJLinyard/marth/falco,JJLinyard/marth/marth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489901,1602510,7564,2023-11-12 07:30:00,sheik,falco,fox,21,20,0,marth,...,1602510/fox/falco,7564/marth/sheik,7564/marth/falco,7564/marth/fox,7564/fox/sheik,7564/fox/falco,7564/fox/fox,7564/falco/sheik,7564/falco/falco,7564/falco/fox
1489902,698885,25647,2023-11-12 07:30:00,captainfalcon,link,kirby,296,20,2,falco,...,698885/kirby/marth,25647/falco/captainfalcon,25647/falco/link,25647/falco/kirby,25647/fox/captainfalcon,25647/fox/link,25647/fox/kirby,25647/marth/captainfalcon,25647/marth/link,25647/marth/kirby
1489903,10565,1602510,2023-11-12 07:30:00,fox,captainfalcon,falco,96,17,7,sheik,...,10565/falco/fox,1602510/sheik/fox,1602510/sheik/captainfalcon,1602510/sheik/falco,1602510/falco/fox,1602510/falco/captainfalcon,1602510/falco/falco,1602510/fox/fox,1602510/fox/captainfalcon,1602510/fox/falco
1489904,698885,1602510,2023-11-12 07:30:00,captainfalcon,link,kirby,296,20,2,sheik,...,698885/kirby/fox,1602510/sheik/captainfalcon,1602510/sheik/link,1602510/sheik/kirby,1602510/falco/captainfalcon,1602510/falco/link,1602510/falco/kirby,1602510/fox/captainfalcon,1602510/fox/link,1602510/fox/kirby


In [ ]:
# NOTE: This cell in particular likes to crash if you interrupt it.
#       Do NOT interrupt it.

features_mini = ['p1_m1_usage', 'p1_m2_usage', 'p1_m3_usage', 'p2_m1_usage', 'p2_m2_usage', 'p2_m3_usage']

def get_entry(row, matchup, df, default_value):
    if row[matchup] not in df.columns:
        return default_value
    else:
        return df.at[row['newest_date'], row[matchup]]

# p1/m1/m1, etc...
for matchup in tqdm.tqdm(matchups):
    player_mains_df[matchup + '_elo'] = player_mains_df[['newest_date', matchup]].apply(lambda row: get_entry(row, matchup, alt2_df, 1500.0), axis=1)
    features_mini.append(matchup + '_elo')

    player_mains_df[matchup + '_updates'] = player_mains_df[['newest_date', matchup]].apply(lambda row: get_entry(row, matchup, alt2_updates_df, 0.0), axis=1)
    features_mini.append(matchup + '_updates')

  0%|          | 0/18 [00:00<?, ?it/s]

: 

In [ ]:
player_mains_df

,p1_id,p2_id,start,p1_m1,p1_m2,p1_m3,p1_m1_usage,p1_m2_usage,p1_m3_usage,p2_m1,...,p1/m3/m3_elo,p2/m1/m1_elo,p2/m1/m2_elo,p2/m1/m3_elo,p2/m2/m1_elo,p2/m2/m2_elo,p2/m2/m3_elo,p2/m3/m1_elo,p2/m3/m2_elo,p2/m3/m3_elo
681,1000,Stelzig,2015-01-11 14:16:13,fox,falco,marth,0,0,0,fox,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0,1500.0,1500.0
682,Flikkflakk,IVP,2015-01-11 14:16:13,fox,falco,marth,0,0,0,fox,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0,1500.0,1500.0
683,Humpe,Yomi,2015-01-11 14:16:13,fox,falco,marth,0,0,0,fox,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0,1500.0,1500.0
684,Thomas,Hao,2015-01-11 14:16:13,fox,falco,marth,0,0,0,fox,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0,1500.0,1500.0
685,12870,JJLinyard,2015-01-11 14:16:13,fox,falco,marth,0,0,0,fox,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0,1500.0,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489901,1602510,7564,2023-11-12 07:30:00,sheik,falco,fox,21,20,0,marth,...,1500.000000,1478.823956,1494.381013,1458.358016,1500.000000,1500.000000,1500.000000,1500.0,1500.0,1500.0
1489902,698885,25647,2023-11-12 07:30:00,captainfalcon,link,kirby,296,20,2,falco,...,1500.000000,1483.310625,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0,1500.0,1500.0
1489903,10565,1602510,2023-11-12 07:30:00,fox,captainfalcon,falco,96,17,7,sheik,...,1500.000000,1500.000000,1500.000000,2049.276440,2253.047942,1829.113241,1811.184670,1500.0,1500.0,1500.0
1489904,698885,1602510,2023-11-12 07:30:00,captainfalcon,link,kirby,296,20,2,sheik,...,1500.000000,1500.000000,1500.000000,1500.000000,1829.113241,1500.000000,1500.000000,1500.0,1500.0,1500.0


## Generate final dataset from above features

In [ ]:
dataset_df = pd.concat([sets_df[features_elo_rd], player_mains_df[features_mini], sets_df[[output]]], axis=1)
dataset_df = dataset_df.astype(float)
dataset_df

In [ ]:
dataset_df.to_pickle(data_path + 'dataset_mini.pkl')